In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\KIIT0001\\OneDrive\\Desktop\\Medical-Chatbot-Generative-AI\\research'

In [3]:
import os 
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\KIIT0001\\OneDrive\\Desktop\\Medical-Chatbot-Generative-AI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract the data from the pdf file
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [7]:
extracted_data = load_pdf_file(data='C:/Users/KIIT0001/OneDrive/Desktop/Medical-Chatbot-Generative-AI/Data')


In [8]:
#extracted_data

In [9]:
#Split the Data into text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5699


In [11]:
#text_chunks

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings

In [19]:
#Download the Embeddings from Hugging Face 
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [20]:
embeddings = download_hugging_face_embeddings()

In [22]:
query_result = embeddings.embed_query("Hello World")
print("Length",len(query_result))

Length 384


In [ ]:
#query_result

[-0.03447732329368591,
 0.031023161485791206,
 0.006734949070960283,
 0.02610895223915577,
 -0.03936203941702843,
 -0.16030243039131165,
 0.06692401319742203,
 -0.006441463716328144,
 -0.04745052009820938,
 0.014758840203285217,
 0.07087533921003342,
 0.055527545511722565,
 0.019193334504961967,
 -0.026251288130879402,
 -0.010109501890838146,
 -0.026940416544675827,
 0.022307397797703743,
 -0.022226659581065178,
 -0.1496926099061966,
 -0.017493031919002533,
 0.007676285691559315,
 0.05435223504900932,
 0.003254441311582923,
 0.031725917011499405,
 -0.08462145179510117,
 -0.029405971989035606,
 0.05159560590982437,
 0.048124030232429504,
 -0.003314853413030505,
 -0.058279167860746384,
 0.041969235986471176,
 0.022210724651813507,
 0.128188818693161,
 -0.022338958457112312,
 -0.011656282469630241,
 0.06292833387851715,
 -0.032876305282115936,
 -0.09122604131698608,
 -0.03117535263299942,
 0.0526995025575161,
 0.047034818679094315,
 -0.0842030942440033,
 -0.030056176707148552,
 -0.0207448

In [45]:
from dotenv import load_dotenv
load_dotenv()

True

In [46]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os



index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-mabbuap.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [61]:
#Embed each chunk and upsert the embeddings into your Pinecone index 
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [40]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
#Embed each chunk and upsert the embeddings into your Pinecode index 
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [41]:
docsearch

In [42]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs = {"k":3})

In [43]:
retrieved_docs = retriever.invoke("What is acne?")

In [44]:
retrieved_docs

[Document(id='1fe2ade7-6a5a-4d23-b561-01cf976d8dcc', metadata={'author': 'Clifford', 'creationdate': '2004-12-28T15:38:25-05:00', 'creator': 'PyPDF', 'enhanced': 'By PDF Enhancer 2.5/Win', 'moddate': '2005-05-04T13:53:15-06:00', 'page': 4.0, 'page_label': '5', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\Users\\KIIT0001\\OneDrive\\Desktop\\Medical-Chatbot-Generative-AI\\Data\\Medical_Book.pdf', 'spdf': '1096', 'total_pages': 599.0}, page_content='A\nAbscess\nAcidophilus\nAcne\nAconite\nAcupressure\nAcupuncture\nAdemetionine\nAdie’s pupil\nAfrican pygeum\nAgastache\nAging\nAIDS\nAlcoholism\nAlexander technique\nAlfalfa\nAlisma\nAllergies\nAllium cepa\nAloe\nAlpha-hydroxy\nAlzheimer’s disease\nAmino acids\nAndrographis\nAndrostenedione\nAnemarrhena\nAnemia\nAngelica root\nAngina\nAnise\nAnkylosing spondylitis\nAnorexia nervosa\nAnthroposophical medicine\nAntioxidants\nAnxiety\nBates method\nBayberry\nBedsores\nBedwetting\nBee pollen\nBehavioral optometry\nBehavioral therapy'),


In [48]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4,max_tokens=500)

In [54]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [55]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [58]:
response = rag_chain.invoke({"input": "What is acne?"})
print(response["answer"])



Acne is a common skin condition characterized by the presence of pimples, blackheads, and whiteheads on the face, chest, and back. It is caused by a combination of factors such as excess oil production, clogged pores, bacteria, and inflammation. Treatment options include topical medications, oral medications, and lifestyle changes.


In [60]:
response = rag_chain.invoke({"input": "What is ronaldo?"})
print(response["answer"])



I don't know.
